# Connery 工具包和工具

使用 Connery 工具包和工具，您可以将 Connery Actions 集成到您的 LangChain agent 中。

## 什么是 Connery？

Connery 是一个开源的 AI 插件基础设施。

使用 Connery，您可以轻松创建具有一组自定义 actions 的插件，并将其无缝集成到您的 LangChain agent 中。
Connery 将负责处理运行时、授权、密钥管理、访问管理、审计日志和其他关键功能等重要方面。

此外，在社区的支持下，Connery 还提供了各种即用型开源插件，以增加便利性。

了解更多关于 Connery 的信息：

- GitHub: https://github.com/connery-io/connery
- 文档: https://docs.connery.io

## 设置

### 安装

您需要安装 `langchain_community` 包才能使用 Connery 工具。

In [ ]:
%pip install -qU langchain-community

### 凭据

要在 LangChain 代理中使用 Connery Actions，您需要做一些准备工作：

1. 使用[快速入门](https://docs.connery.io/docs/runner/quick-start/)指南设置 Connery runner。
2. 安装所有您想在代理中使用的 Action 的插件。
3. 设置环境变量 `CONNERY_RUNNER_URL` 和 `CONNERY_RUNNER_API_KEY`，以便工具包能够与 Connery Runner 通信。

In [ ]:
import getpass
import os

for key in ["CONNERY_RUNNER_URL", "CONNERY_RUNNER_API_KEY"]:
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"Please enter the value for {key}: ")

## 工具包

在下面的示例中，我们创建了一个代理，该代理使用两个 Connery Action 来总结公共网页并将摘要通过电子邮件发送出去：

1. 来自 [Summarization](https://github.com/connery-io/summarization-plugin) 插件的 **Summarize public webpage**（总结公共网页）操作。
2. 来自 [Gmail](https://github.com/connery-io/gmail) 插件的 **Send email**（发送电子邮件）操作。

您可以在此处 [here](https://smith.langchain.com/public/4af5385a-afe9-46f6-8a53-57fe2d63c5bc/r) 查看此示例的 LangSmith 跟踪。

In [1]:
import os

from langchain.agents import AgentType, initialize_agent
from langchain_community.agent_toolkits.connery import ConneryToolkit
from langchain_community.tools.connery import ConneryService
from langchain_openai import ChatOpenAI

# Specify your Connery Runner credentials.
os.environ["CONNERY_RUNNER_URL"] = ""
os.environ["CONNERY_RUNNER_API_KEY"] = ""

# Specify OpenAI API key.
os.environ["OPENAI_API_KEY"] = ""

# Specify your email address to receive the email with the summary from example below.
recepient_email = "test@example.com"

# Create a Connery Toolkit with all the available actions from the Connery Runner.
connery_service = ConneryService()
connery_toolkit = ConneryToolkit.create_instance(connery_service)

# Use OpenAI Functions agent to execute the prompt using actions from the Connery Toolkit.
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    connery_toolkit.get_tools(), llm, AgentType.OPENAI_FUNCTIONS, verbose=True
)
result = agent.run(
    f"""Make a short summary of the webpage http://www.paulgraham.com/vb.html in three sentences
and send it to {recepient_email}. Include the link to the webpage into the body of the email."""
)
print(result)



> Entering new AgentExecutor chain...

Invoking: `CA72DFB0AB4DF6C830B43E14B0782F70` with `{'publicWebpageUrl': 'http://www.paulgraham.com/vb.html'}`


{'summary': 'The author reflects on the concept of life being short and how having children made them realize the true brevity of life. They discuss how time can be converted into discrete quantities and how limited certain experiences are. The author emphasizes the importance of prioritizing and eliminating unnecessary things in life, as well as actively pursuing meaningful experiences. They also discuss the negative impact of getting caught up in online arguments and the need to be aware of how time is being spent. The author suggests pruning unnecessary activities, not waiting to do things that matter, and savoring the time one has.'}
Invoking: `CABC80BB79C15067CA983495324AE709` with `{'recipient': 'test@example.com', 'subject': 'Summary of the webpage', 'body': 'Here is a short summary of the webpage http://www.paulgraham.com/vb.ht

在以下内容中，我将把英文 MDX 翻译成简体中文，同时保持代码和非文本元素的英文原文不变。

---

注意：Connery Action 是一个结构化工具，因此您只能在支持结构化工具的代理中使用它。

## Tool

In [ ]:
import os

from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.connery import ConneryService
from langchain_openai import ChatOpenAI

# Specify your Connery Runner credentials.
os.environ["CONNERY_RUNNER_URL"] = ""
os.environ["CONNERY_RUNNER_API_KEY"] = ""

# Specify OpenAI API key.
os.environ["OPENAI_API_KEY"] = ""

# Specify your email address to receive the emails from examples below.
recepient_email = "test@example.com"

# Get the SendEmail action from the Connery Runner by ID.
connery_service = ConneryService()
send_email_action = connery_service.get_action("CABC80BB79C15067CA983495324AE709")

手动运行操作。

In [ ]:
manual_run_result = send_email_action.run(
    {
        "recipient": recepient_email,
        "subject": "Test email",
        "body": "This is a test email sent from Connery.",
    }
)
print(manual_run_result)

使用 OpenAI Functions Agent 运行此操作。

您可以在 LangSmith 中查看此示例的跟踪记录：[https://smith.langchain.com/public/a37d216f-c121-46da-a428-0e09dc19b1dc/r](https://smith.langchain.com/public/a37d216f-c121-4

In [ ]:
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    [send_email_action], llm, AgentType.OPENAI_FUNCTIONS, verbose=True
)
agent_run_result = agent.run(
    f"Send an email to the {recepient_email} and say that I will be late for the meeting."
)
print(agent_run_result)



> Entering new AgentExecutor chain...

Invoking: `CABC80BB79C15067CA983495324AE709` with `{'recipient': 'test@example.com', 'subject': 'Late for Meeting', 'body': 'Dear Team,\n\nI wanted to inform you that I will be late for the meeting today. I apologize for any inconvenience caused. Please proceed with the meeting without me and I will join as soon as I can.\n\nBest regards,\n[Your Name]'}`


{'messageId': '<d34a694d-50e0-3988-25da-e86b4c51d7a7@gmail.com>'}I have sent an email to test@example.com informing them that you will be late for the meeting.

> Finished chain.
I have sent an email to test@example.com informing them that you will be late for the meeting.


注意：Connery Action 是一个结构化工具，因此您只能在支持结构化工具的代理中使用它。

## API 参考

有关 Connery 所有功能和配置的详细文档，请参阅 API 参考：

- 工具集：https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.connery.toolkit.ConneryToolkit.html
- 工具：https://python.langchain.com/api_reference/community/tools/langchain_community.tools.connery.service.ConneryService.html